In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


# University of Strathclyde -  MSc Artificial Intelligence and Applications
# CS982 - Big Data Technologies
# Assignment 1 - Exploring Data
 File Created first created 9th October 2019 by Barry Smart.

## Stage 4 - Data Wrangling
 The purpose of this notebook is to orchestrate the process of wanrgling the data.
 This has been tackled by completing each of the backlog items classed as "Data Wrangling" and placing them into a logica sequence in the notebook:

 1. Ingestion
 2. Trimming
 3. Filtering
 4. Process Blank Cells
 5. Unpivoting
 6. Add Country Netadata
 7. Add Decade Label
 8. Pivoting
 9. Write To File

### Design Decisions
 - The notebook is structured such that it can be run end to end any time a new data cut becomes available to transform the raw data into a consistent format for downstream analysis.
 - Any heavy blocks of code have been written as functions in a seperate "data_wrangling_functions.py" Python file and imported into this notebook.
 - The introduction of multi-indexing is extremely useful for downstream processing.
 - Other fundamental processing of data to support downstream analysis has been "retrofitted" to this stage of the process, so we do not need to keep repeating elsewhere and can do it once and re-use it accordingly - eg setting up a "Decade" column.

### Stage 4.1 - Ingest Data
 First stage is to load the raw data from CSV files as generated from the World Bank's [open data portal](https://databank.worldbank.org/).
 Two files are of interest:
 - The larger file containing "world development indicators" organised by country and year.
 - A smaller "coutry metadata" file containing classifications for each coutry such as region and income group.


In [1]:
import numpy as np
import pandas as pd
import os
import assignment1.helper_functions as hf


In [2]:
# Read the raw World Bank data from the CSV file
data_path = str(os.getcwd()) + "\\assignment1\\"
raw_worldbank_data = pd.read_csv(data_path + "world_bank_data.csv")


In [3]:
raw_worldbank_country_metadata = pd.read_csv(data_path + "world_bank_country_metadata.csv")

In [4]:
raw_worldbank_country_metadata.loc[raw_worldbank_country_metadata["Code"] == "PRK"]


,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,IMF data dissemination standard,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,2-alpha code,WB-2 code,Table Name,Short Name
103,PRK,Democratic People's Republic of Korea,Low income,East Asia & Pacific,NaN,NaN,Democratic People's Republic of Korean won,2008,"Multiple Indicator Cluster Survey, 2017",NaN,...,NaN,NaN,NaN,NaN,NaN,2016.0,KP,KP,"Korea, Dem. People's Rep.",Dem. People's Rep. Korea


In [5]:
raw_worldbank_data.head(10)



,Country Name,Country Code,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,537777811.111111,548888895.555556,546666677.777778,751111191.111111,800000044.444444,1006666637.77778,...,12439087076.7667,15856574731.4411,17804280538.1102,20001615788.6719,20561054090.3995,20484873230.2111,19907111418.9938,19362642266.6484,20191764940.1602,19362969582.3643
1,Afghanistan,AFG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,..,..,..,..,..,..,...,21.390528405312,14.3624414596815,0.426354792856571,12.7522870825788,5.60074466131904,2.72454336495028,1.45131465458005,2.2603142045464,2.66529204636834,1.03066005804328
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973,9169410,9351441,9543205,9744781,9956320,...,28394813,29185507,30117413,31161376,32269589,33370794,34413603,35383128,36296400,37172386
3,Afghanistan,AFG,"Population, male (% of total population)",SP.POP.TOTL.MA.ZS,51.6793381591819,51.5873111082659,51.5006134882727,51.4191256601559,51.3428636401139,51.2716146671892,...,51.2208289079348,51.1752424002444,51.184216996607,51.2328883530119,51.2999296675334,51.3586756854884,51.392950630598,51.4003323853459,51.388384281721,51.3641532982879
4,Afghanistan,AFG,"Population, female (% of total population)",SP.POP.TOTL.FE.ZS,48.3206618408181,48.4126888917341,48.4993865117273,48.5808743398441,48.6571363598861,48.7283853328108,...,48.7791710920652,48.8247575997556,48.815783003393,48.7671116469881,48.7000703324666,48.6413243145116,48.607049369402,48.5996676146541,48.611615718279,48.6358467017121
5,Afghanistan,AFG,"Life expectancy at birth, female (years)",SP.DYN.LE00.FE.IN,33.314,33.84,34.359,34.866,35.364,35.853,...,61.898,62.459,63,63.514,63.999,64.453,64.877,65.275,65.656,..
6,Afghanistan,AFG,"Life expectancy at birth, male (years)",SP.DYN.LE00.MA.IN,31.718,32.224,32.724,33.216,33.7,34.18,...,59.159,59.683,60.19,60.675,61.133,61.563,61.965,62.343,62.701,..
7,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,32.446,32.962,33.471,33.971,34.463,34.948,...,60.484,61.028,61.553,62.054,62.525,62.966,63.377,63.763,64.13,..
8,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,59.7731938409853,59.8608738790779,58.4580149495439,78.7063875407802,82.0952307131832,101.108304853377,...,438.076034406941,543.303041863931,591.16234645088,641.872033785411,637.16504385598,613.856332882312,578.466352941708,547.228110150363,556.302138508508,520.896602719135
9,Afghanistan,AFG,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,..,..,..,..,..,..,...,18.5153687132188,11.264133238155,-2.6810808487635,8.97487956759649,1.97416862890181,-0.665270517698119,-1.6228866716696,-0.541697283221509,0.08207892887539,-1.35017833585495


### Stage 4.2 - Trimming
 This part of the process will:
 - Trim the last 5 rows from the data set as they do not contain data (could be considered redundant given next step below);


In [6]:
raw_worldbank_data.tail(10)

,Country Name,Country Code,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
21907,World,WLD,"Mortality rate, adult, male (per 1,000 male ad...",SP.DYN.AMRT.MA,371.870167591872,366.620519405922,362.93021625082,351.297410830587,338.708444497898,328.205638930645,...,190.444190345198,187.635427959727,184.823696637169,182.462455416421,180.099127354713,177.298309844609,174.085100929693,173.472867275604,178.318112520384,..
21908,World,WLD,"Mortality rate, adult, female (per 1,000 femal...",SP.DYN.AMRT.FE,297.893994349911,293.223969718727,289.821387467588,281.071338723073,271.804581893746,263.635971813592,...,128.008201054076,124.993294650045,122.227902527935,119.529053980065,117.871409920965,116.391670742655,116.083524344293,115.54781327774,119.911900229058,..
21909,World,WLD,"Mortality rate attributed to unsafe water, uns...",SH.STA.WASH.P5,..,..,..,..,..,..,...,..,..,..,..,..,..,..,11.7590910002673,..,..
21910,World,WLD,Mortality rate attributed to unintentional poi...,SH.STA.POIS.P5,..,..,..,..,..,..,...,..,1.62406434494835,..,..,..,..,1.46411063791079,1.45141154191923,..,..
21911,World,WLD,"Central government debt, total (current LCU)",GC.DOD.TOTL.CN,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
21912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21915,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21916,Last Updated: 10/16/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
trimmed_worldbank_data = raw_worldbank_data.head(-5)

In [8]:
trimmed_worldbank_data.tail(10)


,Country Name,Country Code,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
21902,World,WLD,"Suicide mortality rate, male (per 100,000 male...",SH.STA.SUIC.MA.P5,..,..,..,..,..,..,...,..,14.3265740762378,..,..,..,..,13.6224442620525,13.5057595189264,..,..
21903,World,WLD,"Mortality rate, neonatal (per 1,000 live births)",SH.DYN.NMRT,..,..,..,..,..,..,...,22.8,22.1,21.4,20.8,20.2,19.6,19.1,18.6,18.2,17.7
21904,World,WLD,"Mortality rate, infant (per 1,000 live births)",SP.DYN.IMRT.IN,..,..,..,..,..,..,...,38.4,37.1,35.7,34.5,33.4,32.4,31.4,30.6,29.7,28.9
21905,World,WLD,"Mortality from CVD, cancer, diabetes or CRD be...",SH.DYN.NCOM.ZS,..,..,..,..,..,..,...,..,19.8042936893589,..,..,..,..,18.9966378233586,18.7776692069517,..,..
21906,World,WLD,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,..,..,..,..,..,..,...,53.2,51.3,49,47.1,45.4,43.8,42.4,41.1,39.8,38.6
21907,World,WLD,"Mortality rate, adult, male (per 1,000 male ad...",SP.DYN.AMRT.MA,371.870167591872,366.620519405922,362.93021625082,351.297410830587,338.708444497898,328.205638930645,...,190.444190345198,187.635427959727,184.823696637169,182.462455416421,180.099127354713,177.298309844609,174.085100929693,173.472867275604,178.318112520384,..
21908,World,WLD,"Mortality rate, adult, female (per 1,000 femal...",SP.DYN.AMRT.FE,297.893994349911,293.223969718727,289.821387467588,281.071338723073,271.804581893746,263.635971813592,...,128.008201054076,124.993294650045,122.227902527935,119.529053980065,117.871409920965,116.391670742655,116.083524344293,115.54781327774,119.911900229058,..
21909,World,WLD,"Mortality rate attributed to unsafe water, uns...",SH.STA.WASH.P5,..,..,..,..,..,..,...,..,..,..,..,..,..,..,11.7590910002673,..,..
21910,World,WLD,Mortality rate attributed to unintentional poi...,SH.STA.POIS.P5,..,..,..,..,..,..,...,..,1.62406434494835,..,..,..,..,1.46411063791079,1.45141154191923,..,..
21911,World,WLD,"Central government debt, total (current LCU)",GC.DOD.TOTL.CN,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [9]:
trimmed_worldbank_data.shape


(21912, 63)

### Stage 4.3 - Filtering
 This part of the process will:
 - Filter down to the set of "Series Name" that I am interested in analysing.

 Hypothesis : the following broad classes of data are likely to have the biggest impact on *Life Expectency* and *Population Growth*:
 1. Measures of economic properity:
 2. Access to energy;
 3. Environment measurements;
 4. Access to education and technology;

 Suitable series are now selected from the World Bank Data below.

In [10]:
list_of_series_names = [\
    # The two primary data series that I want to explore are Life Expectancy and Population Growth, so lets select those first:
    "Life expectancy at birth, total (years)",\
    #"Life expectancy at birth, male (years)",\
    #"Life expectancy at birth, female (years)",\
    "Population growth (annual %)",\
    "Population, total",\
    #"Population, male (% of total population)",\
    #"Population, female (% of total population)",\
    #
    # Measures of economic prosperity
    "GDP per capita (current US$)",\
    "Inflation, consumer prices (annual %)",\
    "Market capitalization of listed domestic companies (current US$)",\
    "Tax revenue (% of GDP)",\
    "Merchandise exports (current US$)",\
    #
    # Access to clean, reliable energy
    "Electric power consumption (kWh per capita)",\
    "Energy use (kg of oil equivalent per capita)",\
    "Power outages in firms in a typical month (number)",\
    "Fossil fuel energy consumption (% of total)",\
    "Renewable energy consumption (% of total final energy consumption)",\
    #
    # Environmental measures
    "Urban population growth (annual %)",\
    "Population density (people per sq. km of land area)",\
    #"Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population)",\
    #"Mortality caused by road traffic injury (per 100,000 people)",\
    #"Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population)",\
    "Suicide mortality rate (per 100,000 population)",\
    # 
    # Access to education, healthcare and technology
    "Immunization, DPT (% of children ages 12-23 months)",\
    "Mobile cellular subscriptions (per 100 people)",\
    "Account ownership at a financial institution or with a mobile-money-service provider, young adults (% of population ages 15-24)",\
    "Mortality rate, infant (per 1,000 live births)"]


In [11]:
# Now use this list of Series Name to appply this filter to the data:
filtered_worldbank_data = trimmed_worldbank_data.loc[trimmed_worldbank_data['Series Name'].isin(list_of_series_names)]

In [12]:
filtered_worldbank_data["Series Name"].value_counts()


Suicide mortality rate (per 100,000 population)                                                                                    264
Tax revenue (% of GDP)                                                                                                             264
Mobile cellular subscriptions (per 100 people)                                                                                     264
GDP per capita (current US$)                                                                                                       264
Electric power consumption (kWh per capita)                                                                                        264
Fossil fuel energy consumption (% of total)                                                                                        264
Immunization, DPT (% of children ages 12-23 months)                                                                                264
Merchandise exports (current US$)                      

### Stage 4.4 - Process Blank Cells
 One simple step is required at this stage to clean up the cells that to contain no data : that is to replace the instances of ".." with NaN.


In [13]:
cleansed_world_data = filtered_worldbank_data.replace(to_replace='..', value=np.nan)


### Stage 4.5 - Unpivoting
 This part of the process will:
 - Rename the year columns - a pre-quisite to support the next step;
 - Unpivot the data such that the individual year columns are collapsed into single column to achieve a "thin and tall" data structure;
 - Pivot the data such that the individual series data are each placed into their own columns to achieve a "fatter and less tall" data stucture.


In [14]:
reshaped_worldbank_data, column_list = hf.trim_year_column_names(cleansed_world_data)

In [15]:
reshaped_worldbank_data.shape

(5280, 63)

In [16]:
reshaped_worldbank_data = pd.melt(reshaped_worldbank_data, id_vars=['Series Name', 'Series Code', 'Country Name', 'Country Code'])

In [17]:
reshaped_worldbank_data = reshaped_worldbank_data.rename(columns = { "variable" : "Year"})

In [18]:
reshaped_worldbank_data = reshaped_worldbank_data.astype({"value" : "float"})

In [19]:
reshaped_worldbank_data.shape

(311520, 6)

In [20]:
reshaped_worldbank_data.head(5)



,Series Name,Series Code,Country Name,Country Code,Year,value
0,"Population, total",SP.POP.TOTL,Afghanistan,AFG,1960,8.996973e+06
1,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,Afghanistan,AFG,1960,3.244600e+01
2,GDP per capita (current US$),NY.GDP.PCAP.CD,Afghanistan,AFG,1960,5.977319e+01
3,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,Afghanistan,AFG,1960,NaN
4,Power outages in firms in a typical month (num...,IC.ELC.OUTG,Afghanistan,AFG,1960,NaN


### Stage 4.6 - Add Country Metadata
 Here we prepare the World Bank country metadata

In [21]:
raw_worldbank_country_metadata.columns


Index(['Code', 'Long Name', 'Income Group', 'Region', 'Lending category',
       'Other groups', 'Currency Unit', 'Latest population census',
       'Latest household survey', 'Special Notes',
       'National accounts base year', 'National accounts reference year',
       'System of National Accounts', 'SNA price valuation',
       'Alternative conversion factor', 'PPP survey year',
       'Balance of Payments Manual in use', 'External debt Reporting status',
       'System of trade', 'Government Accounting concept',
       'IMF data dissemination standard',
       'Source of most recent Income and expenditure data',
       'Vital registration complete', 'Latest agricultural census',
       'Latest industrial data', 'Latest trade data', '2-alpha code',
       'WB-2 code', 'Table Name', 'Short Name'],
      dtype='object')

In [22]:
country_metadata = raw_worldbank_country_metadata[["Code", "Short Name", "Income Group", "Region"]]


In [23]:
country_metadata = country_metadata.rename(columns = { "Short Name" : "Country"})


In [24]:
country_metadata.head(10)


,Code,Country,Income Group,Region
0,AFG,Afghanistan,Low income,South Asia
1,ALB,Albania,Upper middle income,Europe & Central Asia
2,DZA,Algeria,Upper middle income,Middle East & North Africa
3,ASM,American Samoa,Upper middle income,East Asia & Pacific
4,AND,Andorra,High income,Europe & Central Asia
5,AGO,Angola,Lower middle income,Sub-Saharan Africa
6,ATG,Antigua and Barbuda,High income,Latin America & Caribbean
7,ARG,Argentina,Upper middle income,Latin America & Caribbean
8,ARM,Armenia,Upper middle income,Europe & Central Asia
9,ABW,Aruba,High income,Latin America & Caribbean


 There are a number of "dummy" countries that have empty Region and Income Group data - these will ultimately be dropped from the data when the pivot stage is applied below.

In [25]:
country_metadata.loc[pd.isnull(country_metadata["Region"])]


,Code,Country,Income Group,Region
217,ARB,Arab World,NaN,NaN
218,CSS,Caribbean small states,NaN,NaN
219,CEB,Central Europe and the Baltics,NaN,NaN
220,EAR,Early-demographic dividend,NaN,NaN
221,EAS,East Asia & Pacific,NaN,NaN
222,EAP,East Asia & Pacific (excluding high income),NaN,NaN
223,TEA,East Asia & Pacific (IDA & IBRD),NaN,NaN
224,EMU,Euro area,NaN,NaN
225,ECS,Europe & Central Asia,NaN,NaN
226,ECA,Europe & Central Asia (excluding high income),NaN,NaN


 Here we will extend the World Bank data with country metadata (Region, Income Group) by merging the two data sets.

In [26]:
merged_data = reshaped_worldbank_data.merge(country_metadata, left_on="Country Code", right_on="Code")


In [27]:
merged_data.head(10)


,Series Name,Series Code,Country Name,Country Code,Year,value,Code,Country,Income Group,Region
0,"Population, total",SP.POP.TOTL,Afghanistan,AFG,1960,8.996973e+06,AFG,Afghanistan,Low income,South Asia
1,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,Afghanistan,AFG,1960,3.244600e+01,AFG,Afghanistan,Low income,South Asia
2,GDP per capita (current US$),NY.GDP.PCAP.CD,Afghanistan,AFG,1960,5.977319e+01,AFG,Afghanistan,Low income,South Asia
3,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,Afghanistan,AFG,1960,NaN,AFG,Afghanistan,Low income,South Asia
4,Power outages in firms in a typical month (num...,IC.ELC.OUTG,Afghanistan,AFG,1960,NaN,AFG,Afghanistan,Low income,South Asia
5,Energy use (kg of oil equivalent per capita),EG.USE.PCAP.KG.OE,Afghanistan,AFG,1960,NaN,AFG,Afghanistan,Low income,South Asia
6,Fossil fuel energy consumption (% of total),EG.USE.COMM.FO.ZS,Afghanistan,AFG,1960,NaN,AFG,Afghanistan,Low income,South Asia
7,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,Afghanistan,AFG,1960,NaN,AFG,Afghanistan,Low income,South Asia
8,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,Afghanistan,AFG,1960,NaN,AFG,Afghanistan,Low income,South Asia
9,Mobile cellular subscriptions (per 100 people),IT.CEL.SETS.P2,Afghanistan,AFG,1960,0.000000e+00,AFG,Afghanistan,Low income,South Asia


In [28]:
merged_data["Country Name"].value_counts()


Germany                  1180
Tunisia                  1180
Micronesia, Fed. Sts.    1180
Nepal                    1180
Bahamas, The             1180
                         ... 
Chad                     1180
Paraguay                 1180
India                    1180
Serbia                   1180
China                    1180
Name: Country Name, Length: 263, dtype: int64

### Stage 1.7 - Add Decade Label
 Here we process the Year colun to create a useful Decade column and convert Year from a string into an integer.

In [29]:
merged_data["Decade"] = merged_data["Year"].str.slice(start=0, stop=3) + "0s"


In [30]:
merged_data = merged_data.astype({"Year" : "int"})


In [31]:
merged_data.head(10)


,Series Name,Series Code,Country Name,Country Code,Year,value,Code,Country,Income Group,Region,Decade
0,"Population, total",SP.POP.TOTL,Afghanistan,AFG,1960,8.996973e+06,AFG,Afghanistan,Low income,South Asia,1960s
1,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,Afghanistan,AFG,1960,3.244600e+01,AFG,Afghanistan,Low income,South Asia,1960s
2,GDP per capita (current US$),NY.GDP.PCAP.CD,Afghanistan,AFG,1960,5.977319e+01,AFG,Afghanistan,Low income,South Asia,1960s
3,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,Afghanistan,AFG,1960,NaN,AFG,Afghanistan,Low income,South Asia,1960s
4,Power outages in firms in a typical month (num...,IC.ELC.OUTG,Afghanistan,AFG,1960,NaN,AFG,Afghanistan,Low income,South Asia,1960s
5,Energy use (kg of oil equivalent per capita),EG.USE.PCAP.KG.OE,Afghanistan,AFG,1960,NaN,AFG,Afghanistan,Low income,South Asia,1960s
6,Fossil fuel energy consumption (% of total),EG.USE.COMM.FO.ZS,Afghanistan,AFG,1960,NaN,AFG,Afghanistan,Low income,South Asia,1960s
7,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,Afghanistan,AFG,1960,NaN,AFG,Afghanistan,Low income,South Asia,1960s
8,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,Afghanistan,AFG,1960,NaN,AFG,Afghanistan,Low income,South Asia,1960s
9,Mobile cellular subscriptions (per 100 people),IT.CEL.SETS.P2,Afghanistan,AFG,1960,0.000000e+00,AFG,Afghanistan,Low income,South Asia,1960s


In [32]:
merged_data.tail(10)


,Series Name,Series Code,Country Name,Country Code,Year,value,Code,Country,Income Group,Region,Decade
310330,Account ownership at a financial institution o...,FX.OWN.TOTL.YG.ZS,World,WLD,2018,NaN,WLD,World,NaN,NaN,2010s
310331,Population density (people per sq. km of land ...,EN.POP.DNST,World,WLD,2018,5.963624e+01,WLD,World,NaN,NaN,2010s
310332,Population growth (annual %),SP.POP.GROW,World,WLD,2018,1.108774e+00,WLD,World,NaN,NaN,2010s
310333,Urban population growth (annual %),SP.URB.GROW,World,WLD,2018,1.935715e+00,WLD,World,NaN,NaN,2010s
310334,Merchandise exports (current US$),TX.VAL.MRCH.CD.WT,World,WLD,2018,1.960045e+13,WLD,World,NaN,NaN,2010s
310335,Market capitalization of listed domestic compa...,CM.MKT.LCAP.CD,World,WLD,2018,6.865409e+13,WLD,World,NaN,NaN,2010s
310336,"Immunization, DPT (% of children ages 12-23 mo...",SH.IMM.IDPT,World,WLD,2018,8.586973e+01,WLD,World,NaN,NaN,2010s
310337,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,World,WLD,2018,NaN,WLD,World,NaN,NaN,2010s
310338,"Suicide mortality rate (per 100,000 population)",SH.STA.SUIC.P5,World,WLD,2018,NaN,WLD,World,NaN,NaN,2010s
310339,"Mortality rate, infant (per 1,000 live births)",SP.DYN.IMRT.IN,World,WLD,2018,2.890000e+01,WLD,World,NaN,NaN,2010s


In [33]:
merged_data.dtypes


Series Name      object
Series Code      object
Country Name     object
Country Code     object
Year              int32
value           float64
Code             object
Country          object
Income Group     object
Region           object
Decade           object
dtype: object

### Stage 1.8 - Pivoting
 This part of the process will:
 1. Pivot the data such that the individual series data are each placed into their own columns to achieve a "fatter and less tall" data stucture;
 2. In doing so, create a useful multi-index that can be used to slice and group data later in the process.


In [34]:
pivoted_worldbank_data = pd.pivot_table(merged_data, index=["Region", "Income Group", "Country", "Decade", "Year"], columns="Series Name", values="value")


In [35]:
# Rename super long column as it is a pain later in the process:
pivoted_worldbank_data = pivoted_worldbank_data.rename(columns = { "Account ownership at a financial institution or with a mobile-money-service provider, young adults (% of population ages 15-24)" \
 : "Account at financial institution (% of population ages 15-24)" })


In [36]:
pivoted_worldbank_data.shape

(12726, 20)

In [37]:
pivoted_worldbank_data.describe()

Series Name,Account at financial institution (% of population ages 15-24),Electric power consumption (kWh per capita),Energy use (kg of oil equivalent per capita),Fossil fuel energy consumption (% of total),GDP per capita (current US$),"Immunization, DPT (% of children ages 12-23 months)","Inflation, consumer prices (annual %)","Life expectancy at birth, total (years)",Market capitalization of listed domestic companies (current US$),Merchandise exports (current US$),Mobile cellular subscriptions (per 100 people),"Mortality rate, infant (per 1,000 live births)",Population density (people per sq. km of land area),Population growth (annual %),"Population, total",Power outages in firms in a typical month (number),Renewable energy consumption (% of total final energy consumption),"Suicide mortality rate (per 100,000 population)",Tax revenue (% of GDP),Urban population growth (annual %)
count,427.000000,5907.000000,6082.000000,5860.000000,9675.000000,6871.000000,7671.000000,11329.000000,2.199000e+03,1.038000e+04,9418.000000,10097.000000,12162.000000,12690.000000,1.269500e+04,259.000000,5392.000000,915.000000,3843.000000,12573.000000
mean,44.536078,3173.076628,2348.631092,66.421132,8172.122518,79.063601,25.078948,64.000279,5.166738e+11,3.172074e+10,30.210696,51.738219,328.717642,1.801935,2.415184e+07,6.579923,30.752026,9.935628,16.980142,2.967728
std,30.015059,4446.376151,3026.611948,30.660445,16101.936489,22.790947,344.911953,11.454812,2.084727e+12,1.262465e+11,47.816411,45.985846,1583.854763,1.673228,1.014191e+08,12.273851,31.071563,7.267047,7.171823,3.035699
min,0.000000,0.000000,0.000000,0.000000,34.790581,1.000000,-18.108630,18.907000,3.040000e+06,0.000000e+00,0.000000,1.400000,0.098625,-10.955149,3.893000e+03,0.000000,0.000000,0.300000,0.000108,-187.141962
25%,18.906853,399.509265,512.679375,43.421139,513.870461,71.000000,2.191332,55.907000,7.169470e+09,1.550000e+08,0.000000,14.600000,20.253278,0.735368,4.628345e+05,0.750000,3.228826,4.900000,11.882260,1.186377
50%,37.984398,1574.391404,1212.125397,76.479407,1864.536803,88.000000,5.040832,67.134000,4.408117e+10,1.120000e+09,0.241671,37.300000,65.731890,1.737298,4.099702e+06,1.900000,18.646600,8.100000,16.406379,2.667568
75%,70.307777,4305.076959,3067.663829,92.886309,7708.915191,96.000000,10.579520,72.618000,2.352539e+11,9.185500e+09,53.541517,77.900000,153.066933,2.680775,1.311979e+07,6.400000,54.072173,12.700000,21.688350,4.326036
max,100.000000,54799.174708,40710.112176,100.000000,189170.895671,99.000000,23773.131774,85.417073,3.212070e+13,2.487045e+12,345.324522,279.400000,21389.100000,28.059956,1.392730e+09,100.700000,98.342602,52.600000,62.858611,48.935720


In [38]:
pivoted_worldbank_data.head(10)


1964                                    87.975807   
                                                  1965                                    88.041991   
                                                  1966                                    89.241769   
                                                  1967                                    89.500850   
                                                  1968                                    90.104615   
                                                  1969                                    90.269620   

Series Name                                             GDP per capita (current US$)  \
Region              Income Group Country   Decade Year                                 
East Asia & Pacific High income  Australia 1960s  1960                   1807.785710   
                                                  1961                   1874.838940   
                                                  1962                   1851.841851   
                                                  1963                   1964.150470   
                                                  1964                   2128.068355   
                                                  1965                   2277.656742   
                                                  1966                   2340.438685   
                                                  1967                   2576.284509   
                                                  1968                   2720.082609   
                                                  1969                   2986.949507   

Series Name                                             Immunization, DPT (% of children ages 12-23 months)  \
Region              Income Group Country   Decade Year                                                        
East Asia & Pacific High income  Australia 1960s  1960                                                NaN     
                                                  1961                                                NaN     
                                                  1962                                                NaN     
                                                  1963                                                NaN     
                                                  1964                                                NaN     
                                                  1965                                                NaN     
                                                  1966                                                NaN     
                                                  1967                                                NaN     
                                                  1968                                                NaN     
                                                  1969                                                NaN     

Series Name                                             Inflation, consumer prices (annual %)  \
Region              Income Group Country   Decade Year                                          
East Asia & Pacific High income  Australia 1960s  1960                               3.728814   
                                                  1961                               2.287582   
                                                  1962                              -0.319489   
                                                  1963                               0.641026   
                                                  1964                               2.866242   
                                                  1965                               3.405573   
                                                  1966                               3.293413   
                                                  1967                               3.478261   
                                                  1968                 

In [39]:
pivoted_worldbank_data.reset_index()["Country"].value_counts()


Australia                    59
Colombia                     59
Belize                       59
Malaysia                     59
India                        59
                             ..
Mexico                       59
Peru                         59
West Bank and Gaza           44
Serbia                       35
Sint Maarten (Dutch part)    21
Name: Country, Length: 217, dtype: int64

In [40]:
pivoted_worldbank_data.dtypes


Series Name
Account at financial institution (% of population ages 15-24)         float64
Electric power consumption (kWh per capita)                           float64
Energy use (kg of oil equivalent per capita)                          float64
Fossil fuel energy consumption (% of total)                           float64
GDP per capita (current US$)                                          float64
Immunization, DPT (% of children ages 12-23 months)                   float64
Inflation, consumer prices (annual %)                                 float64
Life expectancy at birth, total (years)                               float64
Market capitalization of listed domestic companies (current US$)      float64
Merchandise exports (current US$)                                     float64
Mobile cellular subscriptions (per 100 people)                        float64
Mortality rate, infant (per 1,000 live births)                        float64
Population density (people per sq. km of land area) 

### Stage 1.9 - Write To File
 Now we write the resulting data frame to the Pickle file format to preserve all meta data.

In [41]:
pivoted_worldbank_data.to_pickle(data_path + "pivoted_worldbank_data.pkl")
